In [1]:
import os

In [4]:
%pwd

'c:\\Users\\hp\\OneDrive\\Desktop\\ML\\Projects\\NLP\\SummarizeText'

In [3]:
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


In [6]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config



In [ ]:
import os
from transformers import AutoTokenizer
from TextSummarizer.logging import logger
from datasets import load_dataset,load_from_disk


ModuleNotFoundError: No module named 'transformers'

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement datasets (from versions: none)
ERROR: No matching distribution found for datasets


ModuleNotFoundError: No module named 'datasets'

In [ ]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(example_batch):
        input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True)

        with tokenizer.as_target_tokenizer():
            target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True)

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features,batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))        

In [15]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-07-18 00:45:09,141: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2025-07-18 00:45:09,145: INFO: common: yaml file: params.yaml loaded successfully:]
[2025-07-18 00:45:09,149: INFO: common: Created directory at: artifacts:]
[2025-07-18 00:45:09,151: INFO: common: Created directory at: artifacts/data_transformation:]


NameError: name 'DataTransformation' is not defined